# BTF quad scan

In [ ]:
import os
from os.path import join
from collections import Counter
from datetime import datetime
import h5py
import numpy as np
from scipy import optimize as opt
from matplotlib import pyplot as plt
import proplot as pplt

pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'dusk_r'

In [ ]:
def get_quad_id(filename):
    return filename.split('_')[3]

def get_quad_number(filename):
    return int(get_quad_id(filename)[2:])
        
folder = 'Diagnostics/Data/Measurements/2022-04-14/'
filenames = os.listdir(folder)
filenames = [filename for filename in filenames if 'orbit_response' not in filename]
filenames = sorted(filenames, key=get_quad_number, reverse=True)
filenames

## Explore one file 

In [ ]:
filename = filenames[0]
file = h5py.File(join(folder, filename), 'r')

In [ ]:
list(file.keys())

In [ ]:
# Errors and warnings from log
for i in range(file['log'].size):
    if not(file['/log'][i, 'level'] == 'INFO'.encode('utf')):
        timestr = datetime.fromtimestamp(file['/log'][0, 'timestamp']).strftime("%m/%d/%Y, %H:%M:%S")
        print(f"{timestr} {file['log'][i, 'message']}")

# Configuration data
for key in file['/config'].keys():
    print(f"{key}")
    print("--------------")
    for name in file['/config'][key].dtype.names:
        print(f"{name}: {file['config'][key][name]}")
    print()

In [ ]:
data = file['scandata']
print('Data:')
for i, name in enumerate(data.dtype.names):
    print(f'{name} [{data.dtype[i]}]')
print()

slits = [key for key in data.dtype.names if 'PositionSync' in key]
print(f'slits: {slits}')

In [ ]:
for name in data.dtype.names:
    if name == 'timestamp':
        continue
    fig, ax = pplt.subplots(figsize=(4, 2))
    ax.plot(data[name], color='black')
    ax.format(xlabel='Iteration', ylabel=name)
    plt.show()

In [ ]:
# Get the quad id that is being scanned.
quad_id = get_quad_id(filename)

# Cut off the first few points.
_data = data[2:]

quad_currents = np.array(list(Counter(_data[f'{quad_id}_CurrentSet'])))

In [ ]:
def build_data_dict(data):
    n_iterations = max(data['iteration'])
    data_dict = {
        'xvals': [],
        'profiles': [],
        'means': [],
        'stds': [],
    }
    for i in range(1, n_iterations + 1):
        slow = np.abs(_data['Slow02Gt'])
        idx = np.argwhere(_data['iteration'] == i)
        profile = slow[idx][:, 0]
        xx = _data[slits[0]][idx][:, 0]
        if i % 2 == 0:
            # The scanner is moving in the opposite direction.
            profile = np.flip(profile)
            xx = np.flip(xx)
        data_dict['profiles'].append(profile)
        data_dict['xvals'].append(xx)
    
    # Make all arrays the same length (they vary by 1 or 2)
    min_steps = min([len(profile) for profile in data_dict['profiles']])
    for key in data_dict.keys():
        data_dict[key] = [item[:min_steps] for item in data_dict[key]]

    # Estimate mean and std from profiles
    for i in range(n_iterations):
        xx = data_dict['xvals'][i]
        fx = data_dict['profiles'][i]
        N = np.sum(fx)
        mean = np.sum(fx * xx) / N
        std = np.sum(fx * (xx - mean)**2) / N
        data_dict['means'].append(mean)
        data_dict['stds'].append(std)    
        
    for key in data_dict.keys():
        data_dict[key] = np.array(data_dict[key])
        
    return data_dict, n_iterations

In [ ]:
data_dict, n_iterations = build_data_dict(_data)

In [ ]:
fig, ax = pplt.subplots()
for i in range(n_iterations):
    offset = i * 0.003
    ax.plot(
        data_dict['xvals'][i], 
        data_dict['profiles'][i] + offset, 
        color='black'
    )
ax.format(xlabel=slits[0], ylabel='BS34');

In [ ]:
def plot_linear_fit(xdata, ydata, ax=None):
    
    def linear_fit(x, slope, intercept):
        return slope * x + intercept
    
    popt, pcov = opt.curve_fit(linear_fit, xdata, ydata)
    slope, intercept = popt
    ax.plot(xdata, linear_fit(xdata, *popt), color='lightgrey', zorder=0)
    ax.scatter(xdata, ydata, color='black', s=14)
    return slope, intercept

In [ ]:
fig, ax = pplt.subplots()
slope, intercept = plot_linear_fit(quad_currents, data_dict['stds'], ax=ax)
xmin, xmax = ax.get_xlim()
ax.format(
    xlabel=f'{quad_id}_CurrentSet', 
    ylabel='RMS beam size [mm]',
    xlim=(xmin - 0.1, xmax + 0.1)
)
ax.annotate(f'slope = {slope:.3f}', xy=(0.03, 0.97), xycoords='axes fraction', 
            horizontalalignment='left', verticalalignment='top')

In [ ]:
data_dicts = []
for filename in filenames:
    file = h5py.File(join(folder, filename), 'r')
    data = file['scandata']
    _data = data[2:]
    quad_id = get_quad_id(filename)
    quad_currents = np.array(list(Counter(_data[f'{quad_id}_CurrentSet'])))
    print(quad_id, quad_currents)
    data_dict, n_iterations = build_data_dict(data)
    data_dict['quad_id'] = quad_id
    data_dict['quad_currents'] = quad_currents
    data_dicts.append(data_dict)

In [ ]:
fig, axes = pplt.subplots(nrows=3, ncols=4, figwidth=8.0)
axes.format(xlabel=slits[0], ylabel='BS34')
for ax, data_dict in zip(axes, data_dicts):
    ax.annotate(data_dict['quad_id'], xy=(0.03, 0.97), xycoords='axes fraction', 
                horizontalalignment='left', verticalalignment='top')   
    if data_dict['quad_id'] in ['QV06', 'QH05']:
        continue
    for i in range(n_iterations):
        offset = i * 0.003
        ax.plot(
            data_dict['xvals'][i], 
            data_dict['profiles'][i] + offset, 
            color='black', alpha=0.6,
        )
        xmin, xmax = ax.get_xlim()
        if xmax > xmin:
            ax.set_xlim(xmax, xmin)
plt.savefig('_output/profiles')

In [ ]:
fig, axes = pplt.subplots(nrows=3, ncols=4, figwidth=7.0, sharex=False, sharey=True, hspace=6.0)
for i, (ax, data_dict) in enumerate(zip(axes, data_dicts)):
    if data_dict['quad_id'] in ['QV06', 'QH05']:
        continue    
    ax.format(xlabel=f"{data_dict['quad_id']}_CurrentSet")
    slope, intercept = plot_linear_fit(data_dict['quad_currents'], data_dict['stds'], ax=ax)
    ax.annotate(f'slope = {slope:.3f}', xy=(0.03, 0.97), xycoords='axes fraction', 
                horizontalalignment='left', verticalalignment='top')
    xmin, xmax = ax.get_xlim()
    xpad = 0.1
    if xmin > xmax:
        xmin, xmax = xmax, xmin
    ax.format(xlim=(xmin - xpad, xmax + xpad))

ymin, ymax = axes[0].get_ylim()
axes.format(ylim=(ymin, ymax * 1.05), ylabel='RMS beam size [mm]')
plt.savefig('_output/beam_size_vs_quad_current')

In [ ]:
fig, axes = pplt.subplots(nrows=3, ncols=4, figwidth=7.0, sharex=False, sharey=True, hspace=6.0)
for i, (ax, data_dict) in enumerate(zip(axes, data_dicts)):
    if data_dict['quad_id'] in ['QV06', 'QH05']:
        continue    
    ax.format(xlabel=f"{data_dict['quad_id']}_CurrentSet")
    slope, intercept = plot_linear_fit(data_dict['quad_currents'], data_dict['means'], ax=ax)
    ax.annotate(f'slope = {slope:.3f}', xy=(0.03, 0.97), xycoords='axes fraction', 
                horizontalalignment='left', verticalalignment='top')
    xmin, xmax = ax.get_xlim()
    xpad = 0.1
    if xmin > xmax:
        xmin, xmax = xmax, xmin
    ax.format(xlim=(xmin - xpad, xmax + xpad))

ymin, ymax = axes[0].get_ylim()
axes.format(ylim=(ymin, ymax * 1.05), ylabel='Mean [mm]')
plt.savefig('_output/beam_mean_pos_vs_quad_current')